In [2]:
import numpy as np
# import torch
# import torch.nn.functional as F
# from Systems import *
import scipy.linalg as sp

import matplotlib.pyplot as plt

In [3]:
import numpy as np


class Model():
    def __init__(self, init_state, timeStep = 1e-2):
        self.state = init_state
        self.timeStep = timeStep
        self.trueWeights = np.array([0.2314, 0.06918, -0.6245, 0.0095, 0.0214])
        self.lDelta = 1
        self.substeps = 1

    def dynamics(self, state, action):
        delta = self.trueWeights[0]*state[0] + self.trueWeights[1]*state[1] + self.trueWeights[2]*np.abs(state[0])*state[1] + self.trueWeights[3]*np.abs(state[1])*state[1] + self.trueWeights[4]*state[1]**3
        x1dot = self.state[1]
        # print(action, self.lDelta)        
        x2dot = delta + self.lDelta*action
        xdot = np.reshape([x1dot, x2dot],(2,1))
        return xdot

    def simModel(self, action):
        xstep = self.euler(self.dynamics, self.state, action, self.timeStep, self.substeps)
        return xstep

    def applyCntrl(self, action):
        self.state = self.simModel(action)

    def euler(self, sys, state, action, timeStep, substeps):
        k1 = sys(state, action)
        xstep = state + timeStep*k1
        return xstep


class RefModel():
    def __init__(self, init_state, timeStep = 1e-2, nf = 0.2, damping =0.5):
        self.state = init_state
        self.timeStep = timeStep
        self.naturalFreq = nf
        self.damping = damping
        self.lDelta = 1
        self.substeps = 1

    def dynamics(self, state, ref_signal):
        x1dot = self.state[1]
        x2dot = -self.naturalFreq**2*state[0]-2*self.damping*self.naturalFreq*state[1] + self.naturalFreq**2*ref_signal
        xdot = np.reshape([x1dot, x2dot],(2,1))
        return xdot

    def simModel(self, action):
        xstep = self.euler(self.dynamics, self.state, action, self.timeStep, self.substeps)
        return xstep

    def applyCntrl(self, action):
        self.state = self.simModel(action)

    def euler(self, sys, state, action, timeStep, substeps):
        k1 = sys(state, action)
        xstep = state + timeStep*k1
        return xstep

In [6]:
class DMRAC():
    def __init__(self, timeStep = 1e-2, n_dim = 2, m_dim = 1):
        
        self.layers = [2, 10, 10, 10, 1]
        # last layer parameters; design with assumption that basis are fixed
        
        # self.P = np.eye(n_dim)
        self.B = np.array([0, 1])
        self.Gamma = np.eye(self.layers[-2])*2
        self.timeStep = timeStep
        
        naturalFreq = 2
        damping = 0.5
        self.feedbackGainP = -naturalFreq^2
        self.feedbackGainD = -2*damping*naturalFreq
        
        A = np.reshape([0,1,self.feedbackGainP,self.feedbackGainD], (2,2))
        Q = 1*np.eye(2)
        # self.B = np.reshape([0,1],(2,1))
        self.P = sp.solve_lyapunov(A.transpose(),Q)
        
        self.W = np.random.random((self.layers[-2],m_dim))
        self.phi = np.random.random(self.layers[-2]).reshape((self.layers[-2],1))
        
        ## known case(A)
        self.kx = np.array([self.feedbackGainP, self.feedbackGainD]).reshape((2,1))
        self.kr = 1
        
        # internal network parameter
        
        
    def getCntrl(self, state, ref_state, ref_signal):
        ## linear part
        e = state - ref_state
        w_dot = np.matmul(self.Gamma, np.matmul(self.phi, np.matmul(e.T, np.matmul(self.P, self.B))))
        
        self.W = self.W + self.timeStep*w_dot
        
        u = np.matmul(self.kx.T,state) + ref_signal - np.matmul(self.W.T, self.phi)
        return u
        
        
        
        
        
        
        

In [ ]:
init_state = np.array([0, 0]).reshape((2,1))

endTime = 25
startTime = 0
timeStep = 1e-1

model = Model(init_state, timeStep = timeStep)
refModel = Model(init_state, timeStep = timeStep)
agent = DMRAC(timeStep=timeStep)


time_grid = np.arange(startTime, endTime, timeStep)
refSignal = np.sin(time_grid)


ref_state_rec = []
state_rec = []
cntrl_rec = []



for i in range(len(time_grid)):
    state = model.state
    ref_state = refModel.state
    ref_sig = refSignal[i]
    print(i,1)
    action = agent.getCntrl(state, ref_state, ref_sig)
    print(i,2)
    model.applyCntrl(action)
    print(i,3)
    refModel.applyCntrl(ref_sig)
    print(i,4)
    
    state_rec.append(state)
    ref_state_rec.append(ref_state)
    cntrl_rec.append(action)

    
plt.subplot(2,1,1)
plt.plot(time_grid,state_rec)
plot.plot(time_grid, ref_state_rec)

plt.subplot(2,1,2)
plt.plot(time_grid, cntrl_rec)

    
    